<a href="https://colab.research.google.com/github/MatheusCarneir0/ADS/blob/main/Tr%C3%A1fegoAI_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Google Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Bibliotecas principais

In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Usa GPU se disponível, senão CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando dispositivo: {device}')


Usando dispositivo: cpu


###Carregar e preparar os dados

In [37]:
def carregar_e_preparar_dados(caminho_csv):
    # 1) Lê o CSV
    df = pd.read_csv(caminho_csv)

    # 2) Normaliza nomes: tira espaços, deixa minúsculo, substitui espaços por underline
    #    e remove caracteres especiais (exceto letras acentuadas, que \w preserva)
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(' ', '_')
          .str.replace(r'[^\w]', '', regex=True)
    )

    # DEBUG: mostra as colunas pós-normalização
    print("Colunas após limpeza:", df.columns.tolist())

    # 3) Converte 'data' para datetime e 'hora' para inteiro (pode gerar NaN se não for convertível)
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df['hora'] = pd.to_numeric(df['hora'], errors='coerce').astype('Int64')
    df = df.dropna(subset=['hora'])  # remove linhas onde 'hora' não era numérico

    # 4) Cria coluna 'data_hora'
    df['data_hora'] = pd.to_datetime(
        df['data'].dt.strftime('%Y-%m-%d') + ' ' + df['hora'].astype(str) + ':00:00'
    )

    # 5) Converte 'quantidade' para numérico e remove NaNs
    df['quantidade'] = pd.to_numeric(df['quantidade'], errors='coerce')
    df = df.dropna(subset=['quantidade'])

    # 6) Agrupa por data_hora, km e veículo (com acento), somando as quantidades
    #    Aqui usamos exatamente a coluna 'veículo' (o nome resultante após df.columns...)
    #    Se quiser confirmar, verifique df.columns antes de rodar esta linha
    dados_hora = (
        df
        .groupby(['data_hora', 'km', 'veículo'])['quantidade']
        .sum()
        .reset_index()
        .sort_values('data_hora')
    )

    return dados_hora


###Dataset customizado

In [38]:
class TrafegoDataset(Dataset):
    def __init__(self, df, seq_len=24):
        self.seq_len = seq_len
        self.samples = []

        # Para cada par (km, veículo) → atenção ao nome 'veículo'
        for (km, tipo), grupo in df.groupby(['km', 'veículo']):
            valores = grupo['quantidade'].tolist()
            # Se houver pelo menos seq_len+1 pontos, criamos as janelas
            if len(valores) > seq_len:
                for i in range(len(valores) - seq_len):
                    x_seq = valores[i:i+seq_len]    # seq_len horas de input
                    y_next = valores[i+seq_len]     # próxima hora como alvo
                    self.samples.append((x_seq, y_next))
            # Se não houver dados suficientes, pulamos esse grupo

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        # x: lista de seq_len floats → tensor [seq_len, 1]
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(-1)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y


###Modelo LSTM simples

In [39]:
class ModeloLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1):
        super().__init__()
        # LSTM: input_size=1 (uma feature: quantidade hora a hora)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # Camada final: transforma hidden_size → 1 valor de saída
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x tem shape [batch_size, seq_len, input_size]
        out, _ = self.lstm(x)        # out: [batch_size, seq_len, hidden_size]
        out = out[:, -1, :]           # pega o último passo: [batch_size, hidden_size]
        out = self.fc(out)            # [batch_size, 1]
        return out


###Função de treino

In [40]:
def treinar_modelo(df, epocas=10, batch_size=32, seq_len=24):
    # 1) Verifica se o DataFrame está vazio
    if df is None or df.empty:
        print("❌ DataFrame vazio. Abortando treino.")
        return None

    # 2) Cria o dataset e checa se há amostras
    dataset = TrafegoDataset(df, seq_len)
    if len(dataset) == 0:
        print(f"❌ Não há amostras para seq_len={seq_len}.")
        return None

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 3) Instancia o modelo, a loss e o otimizador
    modelo = ModeloLSTM().to(device)
    criterio = nn.MSELoss()
    otimizador = torch.optim.Adam(modelo.parameters(), lr=0.001)

    # 4) Loop de treino
    for ep in range(epocas):
        modelo.train()
        loss_total = 0.0

        for x_batch, y_batch in loader:
            x_batch = x_batch.to(device)  # [batch_size, 24, 1]
            y_batch = y_batch.to(device)  # [batch_size]

            otimizador.zero_grad()
            pred = modelo(x_batch).squeeze()    # [batch_size,1] → squeeze() → [batch_size]
            loss = criterio(pred, y_batch)
            loss.backward()
            otimizador.step()

            loss_total += loss.item()

        print(f'Época {ep+1}/{epocas} — Loss: {loss_total:.4f}')

    return modelo


###Código principal

In [41]:
# 1) Montar o Google Drive (se ainda não montou)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    print("Não está no Colab ou o Drive já está montado.")

# 2) Defina o caminho do arquivo no Drive
CAMINHO = '/content/drive/MyDrive/TráfegoAI/Dados processados/020_CE_formatado_completo.csv'

# 3) Carrega e prepara os dados
df = carregar_e_preparar_dados(CAMINHO)

# 4) Treina o modelo
modelo = treinar_modelo(df, epocas=10, batch_size=32, seq_len=24)

# 5) Se o treino foi bem-sucedido, salva os pesos
if modelo is not None:
    torch.save(modelo.state_dict(), 'modelo_lstm_trafego.pth')
    print("✅ Modelo salvo em 'modelo_lstm_trafego.pth'")
else:
    print("❌ Treino falhou ou dataset não continha amostras suficientes.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colunas após limpeza: ['data', 'hora', 'sentido', 'uf', 'br', 'km', 'ano', 'classe', 'quantidade', 'latitude', 'longitude', 'veículo']
Época 1/10 — Loss: 98238638.9463
Época 2/10 — Loss: 38891768.1128
Época 3/10 — Loss: 26475747.1089
Época 4/10 — Loss: 18213947.6388
Época 5/10 — Loss: 13870399.8671
Época 6/10 — Loss: 10091995.1315
Época 7/10 — Loss: 9736726.2847
Época 8/10 — Loss: 8532206.2834
Época 9/10 — Loss: 8979919.1548
Época 10/10 — Loss: 8491020.9888
✅ Modelo salvo em 'modelo_lstm_trafego.pth'


**Teste**

In [44]:
 # 3) Carrega e prepara os dados
df = carregar_e_preparar_dados(CAMINHO)
# ---------------------------------------------------
#     BLOCO DE DEBUG: INVESTIGANDO O DATAFRAME
# ---------------------------------------------------

# 1. Imprime as primeiras linhas para ver a estrutura
print("=== As 10 primeiras linhas de df ===")
print(df.head(10))

# 2. Quantos registros existem por KM?
print("\n=== Quantidade de registros por km ===")
print(df['km'].value_counts().sort_index().head(10), "… (se aparecer '…', há mais km que não cabem aqui)")

# 3. Quais tipos de veículo existem no total?
print("\n=== Lista completa de tipos de veículo (únicos) ===")
print(df['veículo'].unique())

# 4. Para um KM específico (por exemplo 80.2), veja que tipos de veículo aparecem:
km_exemplo = 80.2
tipos_no_km = df[df['km'] == km_exemplo]['veículo'].unique()
print(f"\n=== Tipos de veículo disponíveis para km = {km_exemplo} ===")
print(tipos_no_km)

# 5. Para cada (km, tipo_veículo), quantos registros há?
print("\n=== Exemplos de contagem de registros por (km,veículo) ===")
counts = df.groupby(['km', 'veículo']).size().reset_index(name='contador')
print(counts.head(10), "…")
# ---------------------------------------------------
#        FIM DO BLOCO DE DEBUG
# ---------------------------------------------------


Colunas após limpeza: ['data', 'hora', 'sentido', 'uf', 'br', 'km', 'ano', 'classe', 'quantidade', 'latitude', 'longitude', 'veículo']
=== As 10 primeiras linhas de df ===
   data_hora    km                                            veículo  \
0 2021-09-01   401    Caminhão + Reboque ou 2 Semirreboques (5 eixos)   
1 2021-09-01   401  Caminhão + Reboque ou Semirreboque + Reboque (...   
2 2021-09-01   401                                         Indefinido   
3 2021-09-01   401                  Veículos de Passeio / Utilitários   
4 2021-09-01   401                Ônibus / Caminhão Simples (2 eixos)   
5 2021-09-01   401                Ônibus / Caminhão Simples (3 eixos)   
6 2021-09-01  80,2     Caminhão + 2 Semirreboques + Reboque (9 eixos)   
7 2021-09-01  80,2  Caminhão + Reboque ou Semirreboque + Reboque (...   
8 2021-09-01  80,2                  Veículos de Passeio / Utilitários   
9 2021-09-01  80,2                Ônibus / Caminhão Simples (3 eixos)   

   quantidade  
0       

In [42]:
# a) Reconstrói a arquitetura
modelo_carregado = ModeloLSTM().to(device)

# b) Carrega os pesos (com map_location para lidar com CPU/GPU)
try:
    modelo_carregado.load_state_dict(torch.load('modelo_lstm_trafego.pth', map_location=device))
    modelo_carregado.eval()
    print("✅ Modelo carregado e pronto para inferência")
except FileNotFoundError:
    print("❌ Arquivo 'modelo_lstm_trafego.pth' não encontrado.")
except Exception as e:
    print("❌ Erro ao carregar o modelo:", e)

# c) Exemplo de previsão (se o modelo foi carregado com sucesso)
if hasattr(modelo_carregado, 'eval'):
    # Escolha um KM e um tipo de veículo existentes no df
    target_km = 80.2

    # MOSTRA ALGUNS TIPOS para você escolher
    print("Tipos de veículo disponíveis (alguns exemplos):", df['veículo'].unique()[:5], "…")

    # Use exatamente um dos valores listados acima
    target_tipo = df['veículo'].unique()[0]
    seq_len = 24

    # Filtra todas as linhas para esse KM e tipo, ordenadas por data_hora
    grupo_filtrado = df[(df['km'] == target_km) & (df['veículo'] == target_tipo)] \
                      .sort_values('data_hora')

    print(f"Total de registros encontrados para KM {target_km}, Tipo '{target_tipo}': {len(grupo_filtrado)}")

    # Pega as últimas 24 horas
    grupo_ultimo = grupo_filtrado.tail(seq_len)
    print(f"Número de linhas após .tail({seq_len}): {len(grupo_ultimo)}")

    if len(grupo_ultimo) == seq_len:
        entrada = torch.tensor(grupo_ultimo['quantidade'].tolist(), dtype=torch.float32) \
                     .unsqueeze(0).unsqueeze(-1).to(device)  # [1,24,1]
        with torch.no_grad():
            previsao = modelo_carregado(entrada).item()
        print(f"Previsão próxima hora (KM {target_km}, Tipo '{target_tipo}'): {previsao:.2f}")
    else:
        print(f"❌ Não há {seq_len} horas completas para KM {target_km}, Tipo '{target_tipo}'.")


✅ Modelo carregado e pronto para inferência
Tipos de veículo disponíveis (alguns exemplos): ['Caminhão + Reboque ou 2 Semirreboques (5 eixos)'
 'Caminhão + Reboque ou Semirreboque + Reboque (6 eixos)' 'Indefinido'
 'Veículos de Passeio / Utilitários' 'Ônibus / Caminhão Simples (2 eixos)'] …
Total de registros encontrados para KM 80.2, Tipo 'Caminhão + Reboque ou 2 Semirreboques (5 eixos)': 0
Número de linhas após .tail(24): 0
❌ Não há 24 horas completas para KM 80.2, Tipo 'Caminhão + Reboque ou 2 Semirreboques (5 eixos)'.
